In [28]:
import requests
from bs4 import BeautifulSoup
from collections import Counter
import pandas as pd
import re
import time
from tqdm import tqdm

In [29]:
data = pd.DataFrame(columns=['title','content','link'])
data

,title,content,link


In [ ]:
for page_num in tqdm(range(1,118)):   # 오늘의 시사 : 1 ~ 117 page / 사이언스 : 1 ~ 8 page
    time.sleep(1)
    
    # 페이지 탐색
    page_url = f"https://kids.donga.com/?page_no={page_num}&ptype=article&psub=news&gbn=01"
    page_response = requests.get(page_url)
    page_html = page_response.content

    # BeautifulSoup을 사용하여 HTML 파싱
    page_soup = BeautifulSoup(page_html, 'html.parser')
    links = [link['href'] for link in page_soup.find_all('a')]
    links = Counter(links)
    links = [link for link in links if 'https://kids.donga.com?ptype=article&no' in link]
    
    for news_link in links:
        time.sleep(0.1)
        # 웹 페이지의 HTML을 가져오기
        news_response = requests.get(news_link)
        news_content = news_response.content
        news_soup = BeautifulSoup(news_content, 'html.parser')

        # 기사 제목 추출
        title = news_soup.find('title').text.strip()
        title = re.sub(r'\[.*?\]','',title)

        # 기사 본문 추출
        news_content = []
        content = news_soup.find_all('p')
        for cont in content:
            c = cont.find_all('span', style='font-family: 돋움, dotum;')
            for cc in c:
                news_text = cc.text
                if len(news_text) < 33:
                    continue
                if not re.compile('[ㄱ-ㅎ가-힣]+').search(news_text[:10]) or not re.compile('[ㄱ-ㅎ가-힣]+').search(news_text[-10:]):
                    continue
                #if "애니멀 사이언스 코너에서는" in news_text: continue
                news_content.append(news_text)

        if not news_content:
            content = news_soup.find_all('div','at_content')
            for cont in content:
                c = cont.find_all('p')
                for cc in c:
                    news_text = cc.text
                    if len(news_text) < 33:
                        continue
                    if not re.compile('[ㄱ-ㅎ가-힣]+').search(news_text[:10]) or not re.compile('[ㄱ-ㅎ가-힣]+').search(news_text[-10:]):
                        continue
                    #if "애니멀 사이언스 코너에서는" in news_text: continue
                    news_content.append(news_text)
        #news_content = " ".join(news_content)
        if title and news_content:
            data = pd.concat([data,pd.DataFrame({'title': [title] , 'content' : [news_content] , 'link' : [news_link]})],ignore_index=True)
        


In [31]:
data

,title,content,link
0,"쌍둥이 판다, 엄마 품에서 완전 자연 포육 시작",[국내에서 최초로 태어난 쌍둥이 판다 자매가 최근 엄마 품으로 완전히 돌아갔어요. ...,https://kids.donga.com?ptype=article&no=202311...
1,132억 년 된 최고령 블랙홀 발견,"[우주 관측을 시작한 이래(일정한 때로부터 지금까지)로, 가장 나이가 많은 블랙홀이...",https://kids.donga.com?ptype=article&no=202311...
2,"880억 원 가치의 모네 그림, 경매에 나온다… 생애 마지막까지 ‘빛’을 그려낸 화가",[약 880억 원. 프랑스 출신의 세계적인 화가 클로드 모네(1840∼1926)의 ...,https://kids.donga.com?ptype=article&no=202311...
3,영국 찰스 3세의 첫 ‘킹스 스피치’,[>> 7일 (영국) 런던의 웨스트민스터 궁의 국회의사당. 찰스 3세 영국 국왕이 ...,https://kids.donga.com?ptype=article&no=202311...
4,"일회용 종이컵·빨대·비닐봉투 계속 쓴다… 정부, 일회용품 규제 없던 일로","[정부가 식당과 카페에서 일회용 종이컵, 플라스틱 빨대, 비닐봉투를 사용하는 것을 ...",https://kids.donga.com?ptype=article&no=202311...
...,...,...,...
356,죽음의 더위에 '폭염 인증샷'?… 더위가 사람 잡는 와중에도 아랑곳 않는 관광행렬,[한 여성이 데스밸리 국립공원 온도계 옆에서 인증 사진을 남기고 있다. 캘리포니아=...,https://kids.donga.com?ptype=article&no=202307...
357,여자 축구 월드컵 16강 갈 수 있을까?,[>> 7월 30일 호주 애들레이드. *여자 월드컵 H조의 대한민국과 모로코의 경기...,https://kids.donga.com?ptype=article&no=202307...
358,분리수거하고 빨래 개는 AI 로봇 공개… 프로그래밍 없이 스스로 학습,[“멸종된 동물을 고르세요”라는 주문에 여러 물건 가운데 공룡 모양의 장난감을 집어...,https://kids.donga.com?ptype=article&no=202307...
359,"영국, 횡단보도 보행 신호등 켜지는 시간 늘린다",[영국 정부가 횡단보도를 건너는 보행자들에게 더 많은 시간을 주기 위해 보행 신호등...,https://kids.donga.com?ptype=article&no=202307...


In [32]:
data.to_csv("어린이동아_오늘의시사2.csv",index=False)

In [33]:
news_response = requests.get("https://kids.donga.com/?ptype=article&no=20230404124053734579&psub=news&gbn=01")
news_content = news_response.content
news_soup = BeautifulSoup(news_content, 'html.parser')

# 기사 제목 추출
title = news_soup.find('title').text.strip()
title = re.sub(r'\[.*?\]','',title)

# 기사 본문 추출
news_content = []
content = news_soup.find_all('p')
for cont in content:
    c = cont.find_all('span', style='font-family: 돋움, dotum;')
    for cc in c:
        news_text = cc.text
        if len(news_text) < 33:
            continue
        if not re.compile('[ㄱ-ㅎ가-힣]+').search(news_text[:10]) or not re.compile('[ㄱ-ㅎ가-힣]+').search(news_text[-10:]):
            continue
        #if "애니멀 사이언스 코너에서는" in news_text: continue
        news_content.append(news_text)

if not news_content:
    content = news_soup.find_all('div','at_content')
    for cont in content:
        c = cont.find_all('p')
        for cc in c:
            news_text = cc.text
            if len(news_text) < 33:
                continue
            if not re.compile('[ㄱ-ㅎ가-힣]+').search(news_text[:10]) or not re.compile('[ㄱ-ㅎ가-힣]+').search(news_text[-10:]):
                continue
            #if "애니멀 사이언스 코너에서는" in news_text: continue
            news_content.append(news_text)

In [34]:
title

'일본 해저 8336m에서 심해어 발견'

In [35]:
news_content

['등 외신은 호주 웨스턴오스트레일리아대 연구진이 일본 남부\r\n이즈',
 '이 심해어는 현재까지 발견된 물고기 가운데 가장 깊은 바다에서 발견된 것으로 기록됐어요',
 '이런 환경에서도 살아남을 수 있는 건 젤리처럼 말랑말랑한 이\r\n물고기의 몸이 압력을 견딜 수 있게 하기 때문']